<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">ESO Science Archive</h1> 
  <h2 style="color: #0281c9; font-weight: bold;">Jupyter Notebooks</h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

## Query by source name/position

This notebook demonstrates how to use the astroquery.eso module to search for and download raw HAWK-I near-infrared observations from the ESO archive, specifically targeting Sgr A*, the supermassive black hole at the center of the Milky Way. It guides users through querying both the main ESO archive table and instrument-specific tables, filtering results based on observation parameters, and retrieving raw, unprocessed data for further calibration and analysis, enabling direct access to ESO’s extensive collection of observational datasets.

<hr style="border:2px solid #0281c9"> </hr>

### Importing and basic usage of astroquery.eso 

In this example, we will create an instance of the ESO class using:

```python
from astroquery.eso import Eso # import the ESO module from astroquery
eso = Eso()  # create an instance of the ESO class
```

This instance will be used to demonstrate various queries to the ESO Archive, including searching for raw data and data products from a range of instruments and sources.

In [1]:
from astroquery.eso import Eso # import the ESO module from astroquery

In [2]:
eso = Eso() # create an instance of the ESO class 

### Performing a Small Cone Search Around Sgr A*

To perform a small cone search of **10 arcseconds** around the source **Sgr A**, we first need to resolve the source name to obtain its **Right Ascension (RA)** and **Declination (Dec)** coordinates. This can be done using an online name resolver such as the ``SkyCoord.from_name()`` method from astropy.coordinates, which allows us to resolve object names into precise RA/Dec coordinates using online astronomical databases. This gives: 

```python
ra = 266.835
dec = -28.38528
```

In coordinate values of degrees in the ICRS (International Celestial Reference System) frame.

In [3]:
from astropy.coordinates import SkyCoord # import the SkyCoord class from the astropy.coordinates module
import astropy.units as u # import the astropy.units module

coords = SkyCoord.from_name('Sgr A*') # create a SkyCoord object from the name of the source
radius = 20 *u.arcmin # set the radius of the search to 20 arcminutes

We use `astroquery.eso` to search for observations of **Sgr A** taken with the **HAWKI** instrument on the **VLT (Very Large Telescope)**.

This search can also be accessed via the following: 

[tap-link]: https://archive.eso.org/tap_obs/sync?REQUEST=doQuery&amp;LANG=ADQL&amp;MAXREC=200&amp;FORMAT=txt&amp;QUERY=SELECT%20*%0Afrom%20ivoa.obscore%0Awhere%20intersects(s_region,%20circle(%27%27,%20266.835,%20-28.38528,%200.1666))=1%0AAND%20instrument_name%20=%20%27HAWKI%27

[sp-link]: https://archive.eso.org/scienceportal/home?data_release_date=*:2025-02-07&pos=266.41682,-29.00782&r=0.016667&fovcorners=266.800868,-29.226927,266.033328,-29.226929,266.034957,-28.787142,266.799236,-28.787141&ins_id=HAWKI&dp_type=IMAGE&sort=dist,-fov,-obs_date&s=P%2fDSS2%2fcolor&f=0.669626&fc=266.799236,-28.787141&cs=J2000&av=true&ac=false&c=9,10,11,12,13,15,16,17,18,19,20,21&ta=RES&dts=true&sdtm=%7b%22IMAGE%22%3atrue%7d&at=266.41682,-29.00782&sr=i

- **TAP** - [see here][tap-link]
- **ESO Science Portal** [see here](sp-link)

In [ ]:
instrument = 'HAWKI'       # set the instrument to HAWKI
ra = coords.ra             # get the right ascension of the source
dec = coords.dec           # get the declination of the source
radius = radius.to('deg')  # convert the radius to degrees

table_instrument = eso.query_instrument(instrument, ra=ra.value, dec=dec.value, radius=radius.value) # query
table_main = eso.query_main(instrument, ra=ra.value, dec=dec.value, radius=radius.value) # query 

TypeError: query_main() takes 1 positional argument but 2 positional arguments (and 3 keyword-only arguments) were given

In [ ]:
eso.query_instrument(instrument) # query the instrument

Signature:
eso.query_main(
    *,
    ra: float = None,
    dec: float = None,
    radius: float = None,
    column_filters=None,
    columns=None,
    print_help=False,
    cache=True,
    **kwargs,
)
Docstring:
Query raw data contained in the ESO archive.

Parameters
----------
column_filters : dict
    Constraints applied to the query.
columns : list of strings
    Columns returned by the query.
open_form : bool
    If `True`, opens in your default browser the query form
    for the requested instrument.
help : bool
    If `True`, prints all the parameters accepted in
    ``column_filters`` and ``columns`` for the requested
    ``instrument``.
cache : bool
    Defaults to True. If set overrides global caching behavior.
    See :ref:`caching documentation <astroquery_cache>`.

Returns
-------
table : `~astropy.table.Table`
    A table representing the data available in the archive for the
    specified instrument, matching the constraints specified in
    ``kwargs``. The number of ro

We can now inspect the results, which are stored as an ``astropy.table``. However note here these results include all data from the science achive, included non-science data... 

In [5]:
table[:3] # print the first 3 rows of the table

access_estsize,access_url,datalink_url,date_obs,dec,dec_pnt,det_chip1id,det_chop_ncycles,det_dit,det_expid,det_ndit,dp_cat,dp_id,dp_tech,dp_type,ecl_lat,ecl_lon,exp_start,exposure,filter_path,gal_lat,gal_lon,grat_path,gris_path,ins_mode,instrument,lambda_max,lambda_min,last_mod_date,mjd_obs,ob_id,ob_name,object,obs_mode,origfile,period,pi_coi,prog_id,prog_title,prog_type,ra,ra_pnt,release_date,s_region,slit_path,target,tel_airm_end,tel_airm_start,tel_alt,tel_ambi_fwhm_end,tel_ambi_fwhm_start,tel_ambi_pres_end,tel_ambi_pres_start,tel_ambi_rhum,tel_az,telescope,tpl_expno,tpl_id,tpl_name,tpl_nexp,tpl_seqno,tpl_start
kB,,,,deg,deg,,,s,,,,,,,deg,deg,,s,,deg,deg,,,,,AA,AA,,d,,,,,,,,,,,deg,deg,,,,,,,deg,arcsec,arcsec,hPa,hPa,,deg,,,,,,,
int64,object,object,object,float64,float64,object,int16,float32,int16,int16,object,object,object,object,float64,float64,object,float32,object,float64,float64,object,object,object,object,float64,float64,object,float32,int32,object,object,object,object,int16,object,object,object,int32,float64,float64,object,object,object,object,float32,float32,float32,float32,float32,float32,float32,float32,float32,object,int32,object,object,int32,int32,object
34570,https://dataportal.eso.org/dataPortal/file/HAWKI.2011-04-11T22:48:44.534,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?HAWKI.2011-04-11T22:48:44.534,2011-04-11T22:48:44.5341,-29.00412,-29.00412,,--,2.0,3638,1,CALIB,HAWKI.2011-04-11T22:48:44.534,IMAGE,FLAT,-5.608188,273.312862,2011-04-11T22:48:44.533Z,2.0,"KS,OPEN",0.095481,359.861974,,,IMG,HAWKI,2307.0,1982.0,2011-07-15T21:22:34.963Z,55662.95,200180255,SkyFlat_1004_BB,FLAT,s,HAWKI_IMG_FLAT_SkyFlats101_0001.fits,--,HAWK-I TEAM,60.A-9283(A),HAWK-I COMMISSIONING,0,266.229585,266.229585,2011-04-11T22:48:44.533Z,POSITION J2000 266.229585 -29.00412,,EFLD1004,1.0,1.0,88.997,-1.0,-1.0,742.78,742.8,14.0,224.999,ESO-VLT-U4,1,HAWKI_img_cal_SkyFlats,Imaging skyflats,20,2,2011-04-11T22:47:57
46156,https://dataportal.eso.org/dataPortal/file/HAWKI.2012-03-04T07:45:18.835,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?HAWKI.2012-03-04T07:45:18.835,2012-03-04T07:45:18.8353,-28.93167,-28.93167,,--,1.6762,15235,5,ACQUISITION,HAWKI.2012-03-04T07:45:18.835,IMAGE,OBJECT,-5.530412,273.103558,2012-03-04T07:45:18.837Z,8.381,"KS,OPEN",-0.046128,0.033414,,,IMG,HAWKI,2307.0,1982.0,2012-03-04T09:01:22.917Z,55990.324,200225767,BURST_GC_K_Holotest,OBJECT,s,HAWKI_ACQ_OBJ064_0001.fits,60,"OBSERVATORY, P",60.A-9800(L),TECHNICAL TIME,0,266.469875,266.469875,2012-03-04T07:45:18.837Z,POSITION J2000 266.469875 -28.93167,,FIELD-NUCLEAR-BULGE,1.607,1.61,38.32,1.48,1.39,741.98,742.0,39.5,288.429,ESO-VLT-U4,1,HAWKI_img_acq_FastPhot,Preset telescope and center field,1,1,2012-03-04T07:40:25
34347,https://dataportal.eso.org/dataPortal/file/HAWKI.2012-03-04T07:46:14.326,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?HAWKI.2012-03-04T07:46:14.326,2012-03-04T07:46:14.3260,-28.93167,-28.93167,,--,0.2316,15236,500,SCIENCE,HAWKI.2012-03-04T07:46:14.326,"IMAGE,BURST",OBJECT,-5.530412,273.103558,2012-03-04T07:46:14.327Z,115.8,"KS,OPEN",-0.046128,0.033414,,,IMG,HAWKI,2307.0,1982.0,2012-03-04T09:01:25.270Z,55990.324,200225767,BURST_GC_K_Holotest,FIELD-NUCLEAR-BULGE,s,HAWKI_IMG_OBS_FAST064_0001.fits,60,"OBSERVATORY, P",60.A-9800(L),TECHNICAL TIME,0,266.469875,266.469875,2012-03-04T07:46:14.327Z,POSITION J2000 266.469875 -28.93167,,FIELD-NUCLEAR-BULGE,1.603,1.603,38.525,1.48,1.48,741.98,741.98,40.0,288.384,ESO-VLT-U4,1,HAWKI_img_obs_FastPhot,Imaging fast photometry with jitter (no offsets,1,2,2012-03-04T07:46:02


To search for only the science data, we need to refine our results using the following ``column_filters={'dp_cat':'SCIENCE'}``
```python
table = eso.query_instrument(instrument, ra=ra.value, dec=dec.value, radius=radius.value, column_filters={'dp_cat':'SCIENCE'}) 
```
or alternatively we could use the could simply filter the results already in ``table``. 

In [6]:
# table = eso.query_instrument(instrument, ra=ra.value, dec=dec.value, radius=radius.value, column_filters={'dp_cat':'SCIENCE'}) 

In [7]:
table = table[table['dp_cat']=="SCIENCE"] # select only the rows with the dp_cat column equal to "SCIENCE"
table[:3] # print the first 3 rows of the table

access_estsize,access_url,datalink_url,date_obs,dec,dec_pnt,det_chip1id,det_chop_ncycles,det_dit,det_expid,det_ndit,dp_cat,dp_id,dp_tech,dp_type,ecl_lat,ecl_lon,exp_start,exposure,filter_path,gal_lat,gal_lon,grat_path,gris_path,ins_mode,instrument,lambda_max,lambda_min,last_mod_date,mjd_obs,ob_id,ob_name,object,obs_mode,origfile,period,pi_coi,prog_id,prog_title,prog_type,ra,ra_pnt,release_date,s_region,slit_path,target,tel_airm_end,tel_airm_start,tel_alt,tel_ambi_fwhm_end,tel_ambi_fwhm_start,tel_ambi_pres_end,tel_ambi_pres_start,tel_ambi_rhum,tel_az,telescope,tpl_expno,tpl_id,tpl_name,tpl_nexp,tpl_seqno,tpl_start
kB,,,,deg,deg,,,s,,,,,,,deg,deg,,s,,deg,deg,,,,,AA,AA,,d,,,,,,,,,,,deg,deg,,,,,,,deg,arcsec,arcsec,hPa,hPa,,deg,,,,,,,
int64,object,object,object,float64,float64,object,int16,float32,int16,int16,object,object,object,object,float64,float64,object,float32,object,float64,float64,object,object,object,object,float64,float64,object,float32,int32,object,object,object,object,int16,object,object,object,int32,float64,float64,object,object,object,object,float32,float32,float32,float32,float32,float32,float32,float32,float32,object,int32,object,object,int32,int32,object
34347,https://dataportal.eso.org/dataPortal/file/HAWKI.2012-03-04T07:46:14.326,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?HAWKI.2012-03-04T07:46:14.326,2012-03-04T07:46:14.3260,-28.93167,-28.93167,,--,0.2316,15236,500,SCIENCE,HAWKI.2012-03-04T07:46:14.326,"IMAGE,BURST",OBJECT,-5.530412,273.103558,2012-03-04T07:46:14.327Z,115.8,"KS,OPEN",-0.046128,0.033414,,,IMG,HAWKI,2307.0,1982.0,2012-03-04T09:01:25.270Z,55990.324,200225767,BURST_GC_K_Holotest,FIELD-NUCLEAR-BULGE,s,HAWKI_IMG_OBS_FAST064_0001.fits,60,"OBSERVATORY, P",60.A-9800(L),TECHNICAL TIME,0,266.469875,266.469875,2012-03-04T07:46:14.327Z,POSITION J2000 266.469875 -28.93167,,FIELD-NUCLEAR-BULGE,1.603,1.603,38.525,1.48,1.48,741.98,741.98,40.0,288.384,ESO-VLT-U4,1,HAWKI_img_obs_FastPhot,Imaging fast photometry with jitter (no offsets,1,2,2012-03-04T07:46:02
34422,https://dataportal.eso.org/dataPortal/file/HAWKI.2012-03-04T07:46:32.781,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?HAWKI.2012-03-04T07:46:32.781,2012-03-04T07:46:32.7808,-28.93167,-28.93167,,--,0.2316,15237,500,SCIENCE,HAWKI.2012-03-04T07:46:32.781,"IMAGE,BURST",OBJECT,-5.530412,273.103558,2012-03-04T07:46:32.780Z,115.8,"KS,OPEN",-0.046128,0.033414,,,IMG,HAWKI,2307.0,1982.0,2012-03-04T09:01:21.797Z,55990.324,200225767,BURST_GC_K_Holotest,FIELD-NUCLEAR-BULGE,s,HAWKI_IMG_OBS_FAST064_0002.fits,60,"OBSERVATORY, P",60.A-9800(L),TECHNICAL TIME,0,266.469875,266.469875,2012-03-04T07:46:32.780Z,POSITION J2000 266.469875 -28.93167,,FIELD-NUCLEAR-BULGE,1.6,1.601,38.594,1.62,1.62,741.98,741.98,40.0,288.369,ESO-VLT-U4,1,HAWKI_img_obs_FastPhot,Imaging fast photometry with jitter (no offsets,1,3,2012-03-04T07:46:21
34284,https://dataportal.eso.org/dataPortal/file/HAWKI.2012-03-04T07:46:51.283,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?HAWKI.2012-03-04T07:46:51.283,2012-03-04T07:46:51.2837,-28.93167,-28.93167,,--,0.2316,15238,500,SCIENCE,HAWKI.2012-03-04T07:46:51.283,"IMAGE,BURST",OBJECT,-5.530412,273.103558,2012-03-04T07:46:51.283Z,115.8,"KS,OPEN",-0.046128,0.033414,,,IMG,HAWKI,2307.0,1982.0,2012-03-04T09:01:23.420Z,55990.324,200225767,BURST_GC_K_Holotest,FIELD-NUCLEAR-BULGE,s,HAWKI_IMG_OBS_FAST064_0003.fits,60,"OBSERVATORY, P",60.A-9800(L),TECHNICAL TIME,0,266.469875,266.469875,2012-03-04T07:46:51.283Z,POSITION J2000 266.469875 -28.93167,,FIELD-NUCLEAR-BULGE,1.598,1.598,38.658,1.62,1.62,741.98,741.98,40.0,288.355,ESO-VLT-U4,1,HAWKI_img_obs_FastPhot,Imaging fast photometry with jitter (no offsets,1,4,2012-03-04T07:46:40


We can check all the column names in the table by printing the ``table.colnames`` attribute to see what else we could filter... 

In [8]:
table.colnames # print the column names of the table

['access_estsize',
 'access_url',
 'datalink_url',
 'date_obs',
 'dec',
 'dec_pnt',
 'det_chip1id',
 'det_chop_ncycles',
 'det_dit',
 'det_expid',
 'det_ndit',
 'dp_cat',
 'dp_id',
 'dp_tech',
 'dp_type',
 'ecl_lat',
 'ecl_lon',
 'exp_start',
 'exposure',
 'filter_path',
 'gal_lat',
 'gal_lon',
 'grat_path',
 'gris_path',
 'ins_mode',
 'instrument',
 'lambda_max',
 'lambda_min',
 'last_mod_date',
 'mjd_obs',
 'ob_id',
 'ob_name',
 'object',
 'obs_mode',
 'origfile',
 'period',
 'pi_coi',
 'prog_id',
 'prog_title',
 'prog_type',
 'ra',
 'ra_pnt',
 'release_date',
 's_region',
 'slit_path',
 'target',
 'tel_airm_end',
 'tel_airm_start',
 'tel_alt',
 'tel_ambi_fwhm_end',
 'tel_ambi_fwhm_start',
 'tel_ambi_pres_end',
 'tel_ambi_pres_start',
 'tel_ambi_rhum',
 'tel_az',
 'telescope',
 'tpl_expno',
 'tpl_id',
 'tpl_name',
 'tpl_nexp',
 'tpl_seqno',
 'tpl_start']

## Downloading datasets from the archive

The returned table has an ``dp_id`` column. It can be used to retrieve the datasets with ``retrieve_data()``.

The file names, returned in ``data_files``, points to the decompressed datasets (without the .Z extension) that have been locally downloaded. 

The default location (in the ``astropy`` cache) of the decompressed datasets can be adjusted by providing a destination keyword in the call to ``retrieve_data()``. By default, if a requested dataset is already found, it is not downloaded again from the archive. To force the retrieval of data that are present in the destination directory, use ``continuation=True`` in the call to ``retrieve_data()``.

In [ ]:
data_files = eso.retrieve_data(table['dp_id'][0]) # download the first data product
# data_files = eso.retrieve_data(table['dp_id'][0], continuation=True) # force the download of all files even if present in the destination directory
# data_files = eso.retrieve_data(table['dp_id'][0], destination='./') # specify the destination directory for the download

INFO: Downloading datasets ... [astroquery.eso.core]
INFO: Downloading 1 files ... [astroquery.eso.core]
INFO: Downloading file 1/1 https://dataportal.eso.org/dataPortal/file/HARPS.2003-06-18T09:06:13.920 to /Users/abarnes/.astropy/cache/astroquery/Eso [astroquery.eso.core]
INFO: Found cached file /Users/abarnes/.astropy/cache/astroquery/Eso/HARPS.2003-06-18T09:06:13.920.fits [astroquery.eso.core]
INFO: Done! [astroquery.eso.core]


<hr style="border:2px solid #0281c9"> </hr>